# 3D two-tracer Zeldovich pipeline

This notebook demonstrates the 3D APIs added for two-tracer marked-field workflows. It mirrors the 1D pipeline but uses `Grid3D`, `GaussianFieldGenerator3D`, and `Zeldovich3D`, while keeping polynomial coefficients fixed per tracer.

Key steps:
- Build a 3D grid and power-spectrum function.
- Generate Hermitian-symmetric 3D noise fields.
- Run the two-tracer batched Zeldovich realization and compute marked-field power spectra.


In [ ]:
import jax
import jax.numpy as jnp

from diff_weighted_fields import Grid3D, GaussianFieldGenerator3D, Zeldovich3D

L = 400.0
N = 32
grid = Grid3D((N, N, N), L, R_clip=0.0)

def pk_func(k, theta):
    A, R, n = theta
    spectrum = A * (k * R) ** n * jnp.exp(-(k * R) ** 2)
    return spectrum + 1e-3 * jnp.max(spectrum)

gen = GaussianFieldGenerator3D(grid, pk_func)
zel = Zeldovich3D(gen, scheme="cic")


In [ ]:
theta = jnp.array([2.0, 4.0 * grid.H[0], 2.0])
D = 1.0
R_smooth = 0.0

# Two tracers with fixed polynomial coefficients (shape: 2 x 4)
C = jnp.array([
    [1.0 / 5.0, 0.0, 0.0, 0.0],
    [1.0 / 5.0, 0.0, 0.0, 0.0],
])

# Batch of Hermitian-symmetric 3D noise fields
keys = jax.random.split(jax.random.PRNGKey(0), 4)
noise_batch = jax.vmap(grid.generate_hermitian_noise)(keys)

pk_marked = zel.make_realization_batch_2T(D, theta, R_smooth, C, noise_batch)
pk_marked.shape
